## Data-driven optimization and decision making - Assignment 3
Juha Reinikainen

Solve any benchmark problems (K=2 and 5, n=10) with ParEGO and LCB. Start with 109
design points. Compare the hypervolume of the solutions after 100 exact function
evaluations.


In [213]:
from sklearn.metrics import r2_score
import numpy as np
from pymoo.factory import get_problem, get_visualization
from scipy.stats import qmc
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from pymoo.core.problem import Problem
from scipy.stats import norm
from desdeo_problem.testproblems.TestProblems import test_problem_builder
from desdeo_emo.EAs.NSGAIII import NSGAIII
from desdeo_problem import variable_builder, ScalarObjective, MOProblem, VectorObjective
from desdeo_tools.utilities.quality_indicator import hypervolume_indicator
from pymoo.factory import get_decomposition
import itertools
import math
from pymoo.factory import get_problem
from desdeo_tools.utilities.fast_non_dominated_sorting import non_dominated
import problems

from typing import Union

In [214]:

def make_lcb(models, problem: MOProblem, beta):
    """
    Create multiobjective problem with evaluation 
    with models on lcb with given beta value
    """
    def comp_lcb(X):
        lcbs = np.zeros((len(X), len(models)))
        for modelI, model in enumerate(models):
            x_mean, x_std = model.predict(X, return_std=True)
            x_var = np.sqrt(x_std)
            lcb = x_mean - beta * x_var
            lcbs[:, modelI] = lcb
        return lcbs

    obj_names = [f"f{i}" for i in range(len(models))]
    objectives = VectorObjective(obj_names, comp_lcb, maximize=False)
    return MOProblem([objectives], problem.variables)


In [215]:
def make_surrogate(models, problem: MOProblem):
    """
    Problem with evaluation using models
    """
    def comp(X):
        #function values
        y = np.zeros((len(X), problem.n_of_objectives))
        for modelI, model in enumerate(models):
            x_mean = model.predict(X, return_std=False)
            y[:, modelI] = x_mean
        return y

    obj_names = [f"f{i}" for i in range(len(models))]
    objectives = VectorObjective(obj_names, comp)
    return MOProblem([objectives], problem.variables)


In [216]:
def create_data(problem: Union[MOProblem,Problem], n_samples, seed):
    """
    Create samples with latinhypercube sampling
    """
    if isinstance(problem, MOProblem):
        n_var = problem.n_of_variables
        lb = problem.get_variable_lower_bounds()
        ub = problem.get_variable_upper_bounds()
    else:
        n_var = problem.n_var
        lb = problem.xl
        ub = problem.xu
    # generate initial sample
    lhs = qmc.LatinHypercube(n_var, seed=seed)
    X = lhs.random(n_samples)
    X = qmc.scale(X, lb,ub)
    if isinstance(problem, MOProblem):
        y = problem.evaluate(X).objectives
        return X, y
    y = problem.evaluate(X)
    return X, y

In [217]:
def optimize_with_lcb(the_real_problem, surrogates, X, y, max_evals):
    """
    Optimize multiobjective problem the_real_problem
    using surrogates with given initial data X,y
    terminates after max_evals amount of true evaluations of
    the_real_problem has been done.
    uses lower confidence bound to choose
    the points to evaluate
    """
    lcbProblem = make_lcb(surrogates, the_real_problem, 1e-5)
    surrogateProblem = make_surrogate(surrogates, dtlz2)

    optimizer_acquisition = NSGAIII(
        lcbProblem, 10, n_iterations=1, n_gen_per_iter=10)
    optimizer = NSGAIII(surrogateProblem, 50,
                        n_iterations=1, n_gen_per_iter=10)

    # initial training of surrogates
    for i, surrogate in enumerate(surrogates):
        # each surrogate with its corresponding objective
        surrogate.fit(X, y[:, i])

    n_evals = 0
    while n_evals < max_evals:
        # while optimizer.continue_evolution():
        #     optimizer.iterate()
        # individuals, solutions = optimizer.end()

        # optimize the acquisition
        while optimizer_acquisition.continue_evolution():
            optimizer_acquisition.iterate()
        individuals, solutions = optimizer_acquisition.end()

        # evaluate chosen ones with real objective
        solutions = the_real_problem.evaluate(individuals).objectives
        n_evals += len(solutions)

        # append evaluated ones to dataset
        X = np.append(X, individuals, axis=0)
        y = np.append(y, solutions, axis=0)

        # retrain the surrogates
        for i, surrogate in enumerate(surrogates):
            surrogate.fit(X, y[:, i])

    # final solution
    while optimizer.continue_evolution():
        optimizer.iterate()
    individuals, solutions = optimizer.end()

    return individuals, solutions


In [218]:
class ParEGO:
    """
    ParEGO algorithm
    Optimize multiobjective problem the_real_problem
    using surrogate trained on
    scalarized objectives with given initial data X,y
    terminates after max_evals amount of true evaluations of
    the_real_problem has been done.
    """
    def __init__(self):
        pass

    def createWeightVectors(self, M, H):
        """
        Creates evenly distributed weight vectors
        in M dimensional space with H fractions 
        """
        #0/H,1/H,...H/H
        fractions = np.linspace(0, 1, H+1)
        N = math.comb(H+M-1, M-1)
        U = np.zeros((N, M))
        ui = 0
        #iterate possible fraction sets and pick those that
        #add up to 1
        for u in itertools.product(fractions, repeat=M):
            if math.isclose(sum(u), 1.0):
                U[ui] = np.array(u)
                ui += 1
        return U

    def scalarize(self, F, w):
        """
        Augmented Tchebycheff function 
        """
        WF = w * F
        WFmax = WF.max(axis=1)
        p = 0.05
        WFsum = WF.sum(axis=1)
        Fscalarized = WFmax + p * WFsum
        return Fscalarized.reshape(-1, 1)

    def newLambda(self, W):
        """
        get uniformly randomly selected weight vector
        """
        i = np.random.randint(0, len(W))
        return W[i]

    def optimize(self, the_real_problem, surrogate, X, y, max_evals, s, seed):
        """
        s: controls how many weight vectors are generated
        """
        optimizer_acquisition = GA(pop_size=100)
        optimizer = GA(pop_size=100)

        W = self.createWeightVectors(y.shape[1], s)

        surrogateProblem = problems.Surrogate(the_real_problem, surrogate)

        for _ in range(max_evals):
            w = self.newLambda(W)
            F = self.scalarize(y, w)
            surrogate.fit(X, F)

            best = minimize(surrogateProblem, optimizer, ("n_gen", 10), seed=seed)

            # optimize the acquisition
            ei = problems.ExpectedImprovement(the_real_problem, surrogate, best.F)
            res = minimize(ei, optimizer_acquisition, ("n_gen", 10), seed=seed)
            x_t = res.X

            #evaluate with real
            y_t = the_real_problem.evaluate(x_t)
            # append evaluated ones to dataset
            X = np.append(X, [x_t], axis=0)
            y = np.append(y, [y_t], axis=0)

        #return nondominated solutions in X
        non_dominated_ones = non_dominated(y)
        return X[non_dominated_ones], y[non_dominated_ones]


In [219]:
seed = 1

# use nsga-iii with lcb to solve problem
dtlz2 = test_problem_builder("DTLZ2", n_of_objectives=2, n_of_variables=10)
dtlz5 = test_problem_builder("DTLZ5", n_of_objectives=5, n_of_variables=10)



In [220]:
max_evals = 100

# surrogates = [GaussianProcessRegressor() for _ in range(2)]
# individuals, solutions = optimize_with_lcb(dtlz2, surrogates, X, y, max_evals)
# nadir = np.max(solutions, axis=0)
# hv = hypervolume_indicator(solutions, nadir)
# print(hv)

parego = ParEGO()
#TODO: X, y gets modified maybe
# 109 initial points
X, y = create_data(get_problem("dtlz2", n_var = 10, n_obj = 2), 109, seed)
individuals, solutions = \
    parego.optimize(get_problem("dtlz2", n_var = 10, n_obj = 2), 
                    GaussianProcessRegressor(), X, y, max_evals, 10, seed)
nadir = np.max(solutions, axis=0)
hv = hypervolume_indicator(solutions, nadir)
print(hv)

1.3201525250935524


| Instance             | HV (ParEGO) | HV (LCB) |
|  --                  | --          | --       |
| DTLZ2 (K = 2, n = 10) |             |          |
| DTLZ5 (K = 5, n = 10) |            |          |